In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
# load data from csv file and json convert to pandas dataframe

In [10]:
# Assuming the data is in a CSV with Port_ID and Trade_History columns
df = pd.read_csv("D:\\Assignment\\Assinment\\TRADES_CopyTr_90D_ROI.csv")

# import ast
import ast


# Function to safely parse JSON-like strings
def safe_json_loads(x):
    try:
        if pd.isna(x) or x.strip() == "":
            return None  # Handle empty values
        return eval(x)  # Convert string to a Python list safely
    except Exception as e:
        print(f"Error parsing JSON: {e}")
        return None  # Handle errors


# Parse JSON column
df["Trade_History"] = df["Trade_History"].apply(safe_json_loads)


# Drop rows with missing or invalid Trade_History
df = df.dropna(subset=["Trade_History"])


# Explode the JSON list into individual rows
trade_df = df.explode("Trade_History").reset_index(drop=True)

# Convert JSON dictionaries to columns
df_final = pd.concat(
    [
        trade_df.drop(["Trade_History"], axis=1),
        trade_df["Trade_History"].apply(pd.Series),
    ],
    axis=1,
)

In [11]:
df_final.head()

,Port_IDs,time,symbol,side,price,fee,feeAsset,quantity,quantityAsset,realizedProfit,realizedProfitAsset,baseAsset,qty,positionSide,activeBuy
0,3925368433214965504,1718899656000,SOLUSDT,BUY,132.53700,-0.994027,USDT,1988.05500,USDT,0.0,USDT,SOL,15.0,LONG,True
1,3925368433214965504,1718899618000,DOGEUSDT,BUY,0.12182,-0.279796,USDT,1398.98088,USDT,0.0,USDT,DOGE,11484.0,LONG,False
2,3925368433214965504,1718899618000,DOGEUSDT,BUY,0.12182,-0.039494,USDT,197.47022,USDT,0.0,USDT,DOGE,1621.0,LONG,False
3,3925368433214965504,1718899616000,DOGEUSDT,BUY,0.12182,-0.008284,USDT,16.56752,USDT,0.0,USDT,DOGE,136.0,LONG,True
4,3925368433214965504,1718899616000,DOGEUSDT,BUY,0.12182,-0.046109,USDT,92.21774,USDT,0.0,USDT,DOGE,757.0,LONG,True


In [13]:
# Convert timestamp
df_final["time"] = pd.to_datetime(df_final["time"])

In [15]:
# create a new column to identify the position type
conditions = [
    (df_final["side"] == "BUY") & (df_final["positionSide"] == "LONG"),
    (df_final["side"] == "SELL") & (df_final["positionSide"] == "LONG"),
    (df_final["side"] == "SELL") & (df_final["positionSide"] == "SHORT"),
    (df_final["side"] == "BUY") & (df_final["positionSide"] == "SHORT"),
]
choices = ["long_open", "long_close", "short_open", "short_close"]
df_final["position_type"] = np.select(conditions, choices, default=None)

In [16]:
df_final.head()

,Port_IDs,time,symbol,side,price,fee,feeAsset,quantity,quantityAsset,realizedProfit,realizedProfitAsset,baseAsset,qty,positionSide,activeBuy,position_type
0,3925368433214965504,1970-01-01 00:28:38.899656,SOLUSDT,BUY,132.53700,-0.994027,USDT,1988.05500,USDT,0.0,USDT,SOL,15.0,LONG,True,long_open
1,3925368433214965504,1970-01-01 00:28:38.899618,DOGEUSDT,BUY,0.12182,-0.279796,USDT,1398.98088,USDT,0.0,USDT,DOGE,11484.0,LONG,False,long_open
2,3925368433214965504,1970-01-01 00:28:38.899618,DOGEUSDT,BUY,0.12182,-0.039494,USDT,197.47022,USDT,0.0,USDT,DOGE,1621.0,LONG,False,long_open
3,3925368433214965504,1970-01-01 00:28:38.899616,DOGEUSDT,BUY,0.12182,-0.008284,USDT,16.56752,USDT,0.0,USDT,DOGE,136.0,LONG,True,long_open
4,3925368433214965504,1970-01-01 00:28:38.899616,DOGEUSDT,BUY,0.12182,-0.046109,USDT,92.21774,USDT,0.0,USDT,DOGE,757.0,LONG,True,long_open


## 1. ydata-profiling (Successor of pandas-profiling)

Description: The official successor of pandas-profiling, now maintained under ydata-profiling. It provides similar functionalities with better performance and support.


In [ ]:
# pip install ydata_profiling

In [17]:
import pandas as pd
from ydata_profiling import ProfileReport


# Generate Report
profile = ProfileReport(df_final, explorative=True)
profile.to_file("pd_prf_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## 2. sweetviz Visualization library

Description: Generates beautiful and detailed visual reports comparing datasets or columns. Faster than Pandas Profiling.


In [18]:
# pip install sweetviz

In [19]:
from ydata_profiling import ProfileReport

report = ProfileReport(df_final)
report.to_file("swt_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## 3. Dataprep

Description: A high-performance alternative to Pandas Profiling with interactive reports and better support for large datasets.


In [ ]:
# pip install dataprep

In [ ]:
# from dataprep.eda import create_report

# report = create_report(df)
# report.show_browser()

## 4. AutoViz

Description: An automatic visualization tool for quick EDA with minimal code.


In [20]:
# pip install Autoviz

In [25]:
from autoviz.AutoViz_Class import AutoViz_Class


AV = AutoViz_Class()
AV.AutoViz(df_final)

    Since nrows is smaller than dataset, loading random sample of 150000 rows into pandas...
Shape of your Data Set loaded: (150000, 16)
#######################################################################################
######################## C L A S S I F Y I N G  V A R I A B L E S  ####################
#######################################################################################
Classifying variables in data set...
    Number of Numeric Columns =  5
    Number of Integer-Categorical Columns =  1
    Number of String-Categorical Columns =  2
    Number of Factor-Categorical Columns =  0
    Number of String-Boolean Columns =  2
    Number of Numeric-Boolean Columns =  0
    Number of Discrete String Columns =  2
    Number of NLP String Columns =  0
    Number of Date Time Columns =  1
    Number of ID Columns =  0
    Number of Columns to Delete =  3
    16 Predictors classified...
        3 variable(s) removed since they were ID or low-information variables
        

,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
Port_IDs,int64,0.000000,0,3672754654734989312.000000,4041860229502600192.000000,Column has 8054 outliers greater than upper bound (4115121280886492672.00) or lower than lower bound(3839136459021180416.00). Cap them or remove them.
time,datetime64[ns],0.000000,32,,,Possible date-time colum: transform before modeling step.
symbol,object,0.000000,0,,,Possible high cardinality column with 154 unique values: Use hash encoding or text embedding to reduce dimension.
side,object,0.000000,0,,,No issue
price,float64,0.000000,NA,0.000165,72105.000000,Column has 27293 outliers greater than upper bound (103.57) or lower than lower bound(-61.93). Cap them or remove them.
fee,float64,0.000000,NA,-108.298808,0.000000,Column has 17985 outliers greater than upper bound (0.25) or lower than lower bound(-0.45). Cap them or remove them.
feeAsset,object,0.000000,0,,,Possible Zero-variance or low information colum: drop before modeling step.
quantity,float64,0.000000,NA,0.006694,432553.110000,"Column has 19395 outliers greater than upper bound (1005.37) or lower than lower bound(-565.18). Cap them or remove them., Column has a high correlation with ['fee']. Consider dropping one of them."
quantityAsset,object,0.000000,0,,,Possible Zero-variance or low information colum: drop before modeling step.
realizedProfit,float64,0.000000,NA,-3269.194444,8186.254058,Column has 33437 outliers greater than upper bound (1.48) or lower than lower bound(-0.89). Cap them or remove them.


Number of All Scatter Plots = 15


[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to C:\Users\ANAS
[nltk_data]    |     KHAN\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\cmudict.zip.
[nltk_data]    | Downloading package gazetteers to C:\Users\ANAS
[nltk_data]    |     KHAN\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\gazetteers.zip.
[nltk_data]    | Downloading package genesis to C:\Users\ANAS
[nltk_data]    |     KHAN\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\genesis.zip.
[nltk_data]    | Downloading package gutenberg to C:\Users\ANAS
[nltk_data]    |     KHAN\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\gutenberg.zip.
[nltk_data]    | Downloading package inaugural to C:\Users\ANAS
[nltk_data]    |     KHAN\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to C:\Users\ANAS
[nltk_data]    |    

Could not draw wordcloud plot for symbol. 
Looks like you are missing some required data for this feature.

To download the necessary data, simply run

    python -m textblob.download_corpora

or use the NLTK downloader to download the missing data: http://nltk.org/data.html
If this doesn't fix the problem, file an issue at https://github.com/sloria/TextBlob/issues.

Could not draw wordcloud plot for baseAsset. 
Looks like you are missing some required data for this feature.

To download the necessary data, simply run

    python -m textblob.download_corpora

or use the NLTK downloader to download the missing data: http://nltk.org/data.html
If this doesn't fix the problem, file an issue at https://github.com/sloria/TextBlob/issues.

All Plots done
Time to run AutoViz = 48 seconds 

 ###################### AUTO VISUALIZATION Completed ########################


,Port_IDs,time,symbol,side,price,fee,feeAsset,quantity,quantityAsset,realizedProfit,realizedProfitAsset,baseAsset,qty,positionSide,activeBuy,position_type
26120,4011626972687000576,1970-01-01 00:28:38.538799,ORDIUSDT,BUY,45.537000,-0.175317,USDT,350.634900,USDT,2.337210,USDT,ORDI,7.700,BOTH,True,None
158126,4008711265867865600,1970-01-01 00:28:37.722648,ETHUSDT,SELL,3810.000000,-0.006858,USDT,34.290000,USDT,0.000000,USDT,ETH,0.009,SHORT,True,short_open
101280,4020204877254599680,1970-01-01 00:28:38.207235,ORDIUSDT,SELL,53.854000,-0.099630,USDT,199.259800,USDT,-3.356698,USDT,ORDI,3.700,LONG,False,long_close
190451,3910983669796782848,1970-01-01 00:28:35.440696,BTCUSDT,SELL,61388.800000,-0.405166,USDT,2025.830400,USDT,23.852400,USDT,BTC,0.033,LONG,True,long_close
16936,3996598927198996481,1970-01-01 00:28:38.802587,INJUSDT,SELL,21.068000,-0.013694,USDT,27.388400,USDT,0.557104,USDT,INJ,1.300,LONG,False,long_close
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136118,3946087913459823360,1970-01-01 00:28:38.937570,STGUSDT,BUY,0.417500,-0.005010,USDT,10.020000,USDT,0.096000,USDT,STG,24.000,BOTH,True,None
67076,3977116548751698176,1970-01-01 00:28:35.629173,SOLUSDT,SELL,148.535000,-0.148535,USDT,297.070000,USDT,0.000000,USDT,SOL,2.000,BOTH,False,None
29139,3956076827719377409,1970-01-01 00:28:37.811796,1000PEPEUSDT,SELL,0.013142,-0.011177,USDT,22.353862,USDT,0.652307,USDT,1000PEPE,1701.000,LONG,False,long_close
207378,3988187872032603905,1970-01-01 00:28:36.603227,LDOUSDT,SELL,2.391400,-0.014348,USDT,28.696800,USDT,0.000000,USDT,LDO,12.000,SHORT,False,short_open


## 5. D-Tale

Description: Provides an interactive UI to explore and visualize pandas DataFrames.


In [ ]:
# pip install D-Tale

In [29]:
df_final.shape

(211277, 16)

In [31]:
import dtale

import dtale.global_state as gstate

gstate.max_rows = 250000  # Increase the limit (adjust as needed)

dtale.show(df_final)

## Which one should you use?

- For detailed EDA reports: ydata-profiling (Pandas Profiling), Sweetviz, Dataprep
- For visualization-heavy EDA: AutoViz, D-Tale
- For large datasets: Dataprep, Sweetviz
